In [14]:
import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from time import time

sns.set(font_scale=1.5)
plt.style.use('ggplot')

In [15]:
def random_adjacency_matrix(n_vertices, n_edges, min_weight, max_weight):
    assert(n_edges <= n_vertices*(n_vertices-1)/2), "Too much edges requested"
    A = np.zeros((n_vertices, n_vertices), dtype=np.int)
    pairs = [(i, j) for j in range(n_vertices) for i in range(j)]
    indices = np.random.choice(range(len(pairs)), size=n_edges, replace=False)

    for index in indices:
        i, j = pairs[index]
        A[i, j] = A[j, i] = np.random.randint(min_weight, max_weight)
    return A

In [23]:
G = nx.from_numpy_matrix(
    random_adjacency_matrix(
        n_vertices=100,
        n_edges=500,
        min_weight=1,
        max_weight=100
    )
)

In [36]:
trials = []

u = np.random.randint(0, 100)

for _ in range(100):
    start_time = time()
    nx.shortest_path(G, source=u, weight='weight', method='dijkstra')
    end_time = time()
    running_time = end_time - start_time
    trials.append((running_time, 'Dijkstra'))
    
for _ in range(100):
    start_time = time()
    nx.shortest_path(G, source=u, weight='weight', method='bellman-ford')
    end_time = time()
    running_time = end_time - start_time
    trials.append((running_time, 'Bellman-Ford'))
    
results = pd.DataFrame(trials, columns=['time', 'method'])

In [33]:
Dj = results[results.method=='Dijkstra'].time.values
BF = results[results.method=='Bellman-Ford'].time.values

In [41]:
print('Djkstra mean running time :', np.mean(Dj))
print('Djkstra std of a running time :', np.std(Dj))
print()
print('Bellman-Ford mean running time :', np.mean(BF))
print('Bellman-Ford std of a running time :', np.std(BF))

Djkstra mean running time : 0.0021396541595458986
Djkstra std of a running time : 0.0005388712310364399

Bellman-Ford mean running time : 0.005531625747680664
Bellman-Ford std of a running time : 0.000908579225062432


In [18]:
pairs = [(i, j) for i in range(10) for j in range(10)]
obstacle_indices = np.random.choice(10*10, size=30, replace=False)
obstacle_cells = [pairs[i] for i in obstacle_indices]

In [19]:
G = nx.grid_2d_graph(10, 10)

for obstacle_cell in obstacle_cells:
    for cell in G.nodes:
        try:
            G.remove_edge(obstacle_cell, cell)
        except:
            continue

In [20]:
u_idx, v_idx = np.random.choice(10*10, size=2, replace=False)
while u_idx in obstacle_indices or v_idx in obstacle_indices:
    u_idx, v_idx = np.random.choice(10*10, size=2, replace=False)
    
u = pairs[u_idx]
v = pairs[v_idx]

In [21]:
def dist(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

running_times = []

for _ in range(100):
    start_time = time()
    nx.astar_path(G, u, v, heuristic=dist)
    end_time = time()
    running_times.append(end_time - start_time)

In [40]:
print('A-star mean running time:', np.mean(running_times))

print('A-star std of a running time:', np.std(running_times))

A-star mean running time: 4.200935363769531e-05
A-star std of a running time: 9.300097272084216e-06
